In [1]:
import os 
from dotenv import load_dotenv

from google.generativeai.types import HarmCategory, HarmBlockThreshold

from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

True

In [4]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY

In [35]:
DEFAULT_CANDIDATE_COUNT = 1
DEFAULT_TEMPERATURE = 1.
DEFAULT_TOP_K = 20
DEFAULT_TOP_P = 0.8
DEFAULT_NUM_OUTPUTS = 5096  # tokens

_safety_setting = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
}

generation_config={
    "candidate_count": DEFAULT_CANDIDATE_COUNT,
    "max_output_tokens": DEFAULT_NUM_OUTPUTS,
    "temperature": DEFAULT_TEMPERATURE,
    "top_k": DEFAULT_TOP_K,
    "top_p": DEFAULT_TOP_P,
    "response_mime_type": "application/json",
}

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", 
                             **generation_config, 
                             safety_settings=_safety_setting)

In [6]:
from typing import List, Dict
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.output_parsers import JsonOutputParser

class NetworkPortgroup(BaseModel):
    allow_promiscuous: bool
    default_gateway: str
    dhcp_range: List[str]
    netmask: str
    pg_name: str
    subnet: str
    vlan_id: int

class Network(BaseModel):
    eth0: Dict

class VMInstance(BaseModel):
    cpu_number: str
    disk_size_gb: str
    gateway: str
    hostname: str
    image: str
    instance: int
    network: Network
    os_version: str
    ram_size: str

    @validator('ram_size', pre=True)
    def convert_ram_size(cls, value):
        return int(value)

class NetworkConfig(BaseModel):
    action: str
    env_id: str
    network_portgroup: Dict[str, NetworkPortgroup]
    resource_id: str
    vm_instance: Dict[str, VMInstance]


In [7]:
json_parser = JsonOutputParser(pydantic_object=NetworkConfig)

In [45]:
template = """
You are an assistant for question-answering tasks.Your task is to recommend for users based on their specifications. Based on information that you have:

Unique_ID,Platform,Distributor,Description,Min_CPU,Min_RAM_GB,Min_Storage_GB,Installed_Software,Image_Version
1,Windows,Windows Server,"A general-purpose VM suitable for Windows-based workloads",2,4,50,"OS:Windows Server 2019, IIS:10.0, SQL Server:2019",2019
2,Linux,Ubuntu,"A lightweight VM for Linux development environments",1,2,20,"OS:Ubuntu 20.04, Apache:2.4, MySQL:8.0",20.04
3,Windows,Windows Pro,"High-performance VM for Windows desktop applications",4,8,100,"OS:Windows 10 Pro, Office:2019, Visual Studio:2019",win10_pro
4,Linux,CentOS,"Stable and secure VM for server applications",2,4,50,"OS:CentOS 8, Nginx:1.18, PostgreSQL:13",centos_8
5,MacOS,MacOS,"High-end VM for macOS development and testing",4,16,200,"OS:macOS Catalina, Xcode:12.4, Homebrew:2.7",10.15
6,Linux,Ubuntu,"Compute-optimized VM with a high CPU-to-memory ratio",4,8,100,"OS:Ubuntu 20.04, Docker:20.10, Kubernetes:1.20",20.04
7,Windows,Windows Server,"High memory capacity VM for in-memory databases on Windows",4,16,200,"OS:Windows Server 2019, Redis:6.0, MongoDB:4.4",2019
8,Linux,CentOS,"Storage-optimized VM for data warehousing",2,8,500,"OS:CentOS 8, Hadoop:3.2, Spark:3.0",centos_8_storage
9,Windows,Windows Pro,"Graphics-optimized VM for GPU-accelerated Windows applications",8,32,1000,"OS:Windows 10 Pro, CUDA:11.2, TensorFlow:2.4",win10_pro_gpu
10,MacOS,MacOS,"VM for macOS graphics and video editing",8,32,1000,"OS:macOS Catalina, Final Cut Pro:10.5, Adobe Creative Suite:2020",macos_10.15
11,Linux,Ubuntu,"Web application server optimized for hosting web apps",2,4,50,"OS:Ubuntu 20.04, Apache:2.4, PHP:7.4, MySQL:8.0",20.04_web
12,Linux,CentOS,"Database server optimized for high-performance databases",4,16,100,"OS:CentOS 8, MySQL:8.0, PostgreSQL:13",8
13,Windows,Windows Server,"Firewall server for network security and traffic monitoring",2,4,50,"OS:Windows Server 2019, Windows Defender:10.0, Firewall Software:2.0",2019
14,Linux,Ubuntu,"Firewall server for network security and traffic monitor3ing",2,4,50,"OS:Ubuntu 20.04, UFW:0.36, FirewallD:0.8",20.04_firewall
15,Windows,Windows Server,"High-performance VM for Windows-based web applications",4,8,100,"OS:Windows Server 2019, IIS:10.0, ASP.NET:4.8",2019_server
16,Linux,CentOS,"Database server optimized for big data applications",4,16,200,"OS:CentOS 8, Cassandra:3.11, HBase:2.2",centos_8_database

Question: {question} 

Your answer must be in JSON format. Take this an example: 

{json_data}

"""

json_data = """
{
"action": "create",
"env_id": "9876abdc5432",
"network_portgroup": {
"pg1": {
"allow_promiscuous": true,
"default_gateway": "192.168.1.1",
"dhcp_range": [
"192.168.1.100",
"192.168.1.200"
],
"netmask": "24",
"pg_name": "pg1",
"subnet": "192.168.1.0",
"vlan_id": 10
},
"pg2": {
"allow_promiscuous": true,
"default_gateway": "10.0.0.1",
"dhcp_range": [
"10.0.0.100",
"10.0.0.200"
],
"netmask": "24",
"pg_name": "pg2",
"subnet": "10.0.0.0",
"vlan_id": 20
}
},
"resource_id": "abcdef123456",
"vm_instance": {
"vm1": {
"cpu_number": "4",
"disk_size_gb": "100",
"gateway": "192.168.1.1",
"hostname": "web-server",
"image": "ubuntu",
"instance": 1,
"network": {
"eth0": {
"ipv4_address": [
"192.168.1.10"
],
"ipv4_netmask": ["24"
],
"portgroup": "pg1"
}
},
"os_version": "20.04",
"ram_size": "8192"
},
"vm2": {
"cpu_number": "2",
"disk_size_gb": "50",
"gateway": "10.0.0.1",
"hostname": "db-server",
"image": "centos",
"instance": 1,
"network": {
"eth0": {
"ipv4_address": [
"10.0.0.20"
],
"ipv4_netmask": [
"24"
],
"portgroup": "pg2"
}
},
"os_version": "8",
"ram_size": "4096"
}
}
}
"""
rag_prompt = PromptTemplate.from_template(template, 
                                            partial_variables={"format_instructions": json_parser.get_format_instructions()})

In [46]:
def dict_to_json(content: dict): 
    import json
    content = json.dumps(content)
    return content

In [47]:
rag_chain = (
    {"json_data": lambda x: json_data, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | json_parser
)

In [48]:
ex_00 = "I want a network diagram that showcases how our network is designed to scale and handle increasing traffic. It should depict how our load balancers are set up and how they distribute traffic across multiple servers. I'd like to see the bandwidth of each connection and the capacity of our core network devices to understand any potential bottlenecks. The diagram should also clearly show how our network is segmented to optimize performance for specific applications, like our web server or database cluster."

response = rag_chain.invoke(ex_00)

type(response)

dict

In [49]:
dict_to_json(response)

'{"action": "create", "env_id": "network_diagram", "network_portgroup": {"web_tier": {"allow_promiscuous": true, "default_gateway": "192.168.1.1", "dhcp_range": ["192.168.1.100", "192.168.1.200"], "netmask": "24", "pg_name": "web_tier", "subnet": "192.168.1.0", "vlan_id": 10}, "database_tier": {"allow_promiscuous": true, "default_gateway": "10.0.0.1", "dhcp_range": ["10.0.0.100", "10.0.0.200"], "netmask": "24", "pg_name": "database_tier", "subnet": "10.0.0.0", "vlan_id": 20}}, "resource_id": "network_design", "vm_instance": {"web_server_1": {"cpu_number": "4", "disk_size_gb": "100", "gateway": "192.168.1.1", "hostname": "web-server-1", "image": "ubuntu", "instance": 1, "network": {"eth0": {"ipv4_address": ["192.168.1.10"], "ipv4_netmask": ["24"], "portgroup": "web_tier"}}, "os_version": "20.04", "ram_size": "8192"}, "web_server_2": {"cpu_number": "4", "disk_size_gb": "100", "gateway": "192.168.1.1", "hostname": "web-server-2", "image": "ubuntu", "instance": 2, "network": {"eth0": {"ipv